In [1]:
import numpy as np
import pandas as pd
from linearmodels import PanelOLS
from linearmodels import RandomEffects
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
# data = pd.read_csv('DiD_data.csv')
data = pd.read_csv('DiD_reduced.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41442 entries, 0 to 41441
Data columns (total 17 columns):
rssd9001                    41442 non-null int64
rssd9999                    41442 non-null int64
bhc_avgtradingratio         41442 non-null float64
treat_3_b_avg               41442 non-null int64
after_DFA_1                 41442 non-null int64
dep_roa1                    41328 non-null float64
dep_leverage                41407 non-null float64
dep_lnassets                41442 non-null float64
dep_creditrisk_total3       41426 non-null float64
dep_cir                     41362 non-null float64
dep_depositratio            41440 non-null float64
dep_loans_REratio           41426 non-null float64
dep_liquidity               40140 non-null float64
dep_cpp_bankquarter         41442 non-null int64
affect                      41442 non-null float64
affected_bhc                41442 non-null int64
after_DFA_1*affected_bhc    41442 non-null int64
dtypes: float64(10), int64(7)
memory u

In [4]:
data.head()

,rssd9001,rssd9999,bhc_avgtradingratio,treat_3_b_avg,after_DFA_1,dep_roa1,dep_leverage,dep_lnassets,dep_creditrisk_total3,dep_cir,dep_depositratio,dep_loans_REratio,dep_liquidity,dep_cpp_bankquarter,affect,affected_bhc,after_DFA_1*affected_bhc
0,1020180,20040930,0.0,0,0,0.002772,0.081957,15.601202,0.013304,0.463811,0.561805,0.593738,0.024337,0,0.0,0,0
1,1020180,20041231,0.0,0,0,0.003045,0.082480,15.630583,0.009732,0.456392,0.557617,0.601763,0.025446,0,0.0,0,0
2,1020180,20050331,0.0,0,0,0.002616,0.082074,15.644925,0.011830,0.444011,0.556980,0.600700,0.025153,0,0.0,0,0
3,1020180,20050630,0.0,0,0,0.002647,0.081712,15.679702,0.013654,0.433771,0.571642,0.601042,0.023670,0,0.0,0,0
4,1020180,20050930,0.0,0,0,0.002867,0.082944,15.661868,0.012456,0.400985,0.577408,0.581438,0.029793,0,0.0,0,0


In [5]:
date = pd.Categorical(data.rssd9999)
company_code = pd.Categorical(data.rssd9001)
data = data.set_index(['rssd9001', 'rssd9999'])
data['rssd9999'] = date
data['rssd9001'] = company_code
data['after_DFA_1*affect'] = data.after_DFA_1 * data.affect
data.head()

bhc_avgtradingratio  treat_3_b_avg  after_DFA_1  dep_roa1  \
rssd9001 rssd9999                                                              
1020180  20040930                  0.0              0            0  0.002772   
         20041231                  0.0              0            0  0.003045   
         20050331                  0.0              0            0  0.002616   
         20050630                  0.0              0            0  0.002647   
         20050930                  0.0              0            0  0.002867   

                   dep_leverage  dep_lnassets  dep_creditrisk_total3  \
rssd9001 rssd9999                                                      
1020180  20040930      0.081957     15.601202               0.013304   
         20041231      0.082480     15.630583               0.009732   
         20050331      0.082074     15.644925               0.011830   
         20050630      0.081712     15.679702               0.013654   
         20050930      0.082944     15.661868               0.012456   

                    dep_cir  dep_depositratio  dep_loans_REratio  \
rssd9001 rssd9999                                                  
1020180  20040930  0.463811          0.561805           0.593738   
         20041231  0.456392          0.557617           0.601763   
         20050331  0.444011          0.556980           0.600700   
         20050630  0.433771          0.571642           0.601042   
         20050930  0.400985          0.577408           0.581438   

                   dep_liquidity  dep_cpp_bankquarter  affect  affected_bhc  \
rssd9001 rssd9999                                                             
1020180  20040930       0.024337                    0     0.0             0   
         20041231       0.025446                    0     0.0             0   
         20050331       0.025153                    0     0.0             0   
         20050630       0.023670                    0     0.0             0   
         20050930       0.029793                    0     0.0             0   

                   after_DFA_1*affected_bhc  rssd9999 rssd9001  \
rssd9001 rssd9999                                                
1020180  20040930                         0  20040930  1020180   
         20041231                         0  20041231  1020180   
         20050331                         0  20050331  1020180   
         20050630                         0  20050630  1020180   
         20050930                         0  20050930  1020180   

                   after_DFA_1*affect  
rssd9001 rssd9999                      
1020180  20040930                 0.0  
         20041231                 0.0  
         20050331                 0.0  
         20050630                 0.0  
         20050930                 0.0

In [6]:
# List of Independent Variables
DFA = ['after_DFA_1']
control_vars_w_DFA = ['after_DFA_1','dep_roa1', 'dep_leverage', 'dep_lnassets', 'dep_creditrisk_total3', 'dep_cir', 'dep_depositratio', 'dep_loans_REratio', 'dep_liquidity', 'dep_cpp_bankquarter']
all_control = ['after_DFA_1', 'affect', 'after_DFA_1*affect','dep_roa1', 'dep_leverage', 'dep_lnassets', 'dep_creditrisk_total3', 'dep_cir', 'dep_depositratio', 'dep_loans_REratio', 'dep_liquidity', 'dep_cpp_bankquarter']
control_vars_w_DFA_Affect = ['after_DFA_1*affect','dep_roa1', 'dep_leverage', 'dep_lnassets', 'dep_creditrisk_total3', 'dep_cir', 'dep_depositratio', 'dep_loans_REratio', 'dep_liquidity', 'dep_cpp_bankquarter']

exog_DFA = sm.add_constant(data[DFA]) 
exog_w_DFA = sm.add_constant(data[control_vars_w_DFA])
exog_all = sm.add_constant(data[all_control])
exog_DFA_and_Affect = sm.add_constant(data[control_vars_w_DFA_Affect])

/Users/DavidRaj/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


## Baseline Test Naïve Model with NO FE and NO controls (1)

In [7]:
mod = PanelOLS(data.bhc_avgtradingratio, exog_DFA, entity_effects=False)
re_res = mod.fit()
print(re_res)

                           PanelOLS Estimation Summary                           
Dep. Variable:     bhc_avgtradingratio   R-squared:                        0.0002
Estimator:                    PanelOLS   R-squared (Between):              0.0003
No. Observations:                41442   R-squared (Within):              -0.0032
Date:                 Sun, Nov 03 2019   R-squared (Overall):              0.0002
Time:                         21:47:56   Log-likelihood                 1.033e+05
Cov. Estimator:             Unadjusted                                           
                                         F-statistic:                      6.4188
Entities:                         2473   P-value                           0.0113
Avg Obs:                        16.758   Distribution:                 F(1,41440)
Min Obs:                        1.0000                                           
Max Obs:                        40.000   F-statistic (robust):             6.4188
                

## Baseline Test Naïve Model with NO FE and WITH controls (2)

In [8]:
mod = PanelOLS(data.bhc_avgtradingratio, exog_w_DFA, entity_effects=False)
re_res = mod.fit()
print(re_res)

                           PanelOLS Estimation Summary                           
Dep. Variable:     bhc_avgtradingratio   R-squared:                        0.2340
Estimator:                    PanelOLS   R-squared (Between):              0.2292
No. Observations:                40026   R-squared (Within):              -0.1280
Date:                 Sun, Nov 03 2019   R-squared (Overall):              0.2340
Time:                         21:47:57   Log-likelihood                  1.06e+05
Cov. Estimator:             Unadjusted                                           
                                         F-statistic:                      1222.2
Entities:                         2428   P-value                           0.0000
Avg Obs:                        16.485   Distribution:                F(10,40015)
Min Obs:                        1.0000                                           
Max Obs:                        38.000   F-statistic (robust):             1222.2
                

/Users/DavidRaj/anaconda3/lib/python3.7/site-packages/linearmodels/utility.py:478: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


## Baseline Test Naïve Model with NO FE and WITH controls (3)

In [9]:
mod = PanelOLS(data.bhc_avgtradingratio, exog_all, entity_effects=False)
re_res = mod.fit()
print(re_res)

                           PanelOLS Estimation Summary                           
Dep. Variable:     bhc_avgtradingratio   R-squared:                        0.9018
Estimator:                    PanelOLS   R-squared (Between):              0.9810
No. Observations:                40026   R-squared (Within):               0.0750
Date:                 Sun, Nov 03 2019   R-squared (Overall):              0.9018
Time:                         21:47:58   Log-likelihood                 1.471e+05
Cov. Estimator:             Unadjusted                                           
                                         F-statistic:                   3.061e+04
Entities:                         2428   P-value                           0.0000
Avg Obs:                        16.485   Distribution:                F(12,40013)
Min Obs:                        1.0000                                           
Max Obs:                        38.000   F-statistic (robust):          3.061e+04
                

## Baseline Test Naïve Model WITH FE and WITH controls (4)

In [10]:
mod = PanelOLS(data.bhc_avgtradingratio, exog_DFA_and_Affect, entity_effects=True)
re_res = mod.fit()
print(re_res)

                           PanelOLS Estimation Summary                           
Dep. Variable:     bhc_avgtradingratio   R-squared:                        0.0820
Estimator:                    PanelOLS   R-squared (Between):             -0.1603
No. Observations:                40026   R-squared (Within):               0.0820
Date:                 Sun, Nov 03 2019   R-squared (Overall):             -0.1723
Time:                         21:47:59   Log-likelihood                 1.524e+05
Cov. Estimator:             Unadjusted                                           
                                         F-statistic:                      335.76
Entities:                         2428   P-value                           0.0000
Avg Obs:                        16.485   Distribution:                F(10,37588)
Min Obs:                        1.0000                                           
Max Obs:                        38.000   F-statistic (robust):             335.76
                

## Fixed Effect Models with Matched Sample

In [12]:
data = pd.read_csv('DiD_reduced_Q1_matched_all.csv')
data.head()

,Unnamed: 0,rssd9001,rssd9999,bhc_avgtradingratio,treat_3_b_avg,after_DFA_1,dep_roa1,dep_leverage,dep_lnassets,dep_creditrisk_total3,dep_cir,dep_depositratio,dep_loans_REratio,dep_liquidity,dep_cpp_bankquarter,affect,affected_bhc,after_DFA_1*affected_bhc
0,38,1020201,20040930,0.126453,1,0,0.002906,0.070028,18.610798,0.013813,0.339097,0.390303,0.758043,0.044461,0,0.134151,1,0
1,39,1020201,20041231,0.139026,1,0,0.002053,0.074122,18.764626,0.014689,0.414206,0.358281,0.647956,0.042322,0,0.134151,1,0
2,40,1020201,20050331,0.130043,1,0,0.002237,0.076953,18.768549,0.015002,0.357776,0.357660,0.651144,0.045373,0,0.134151,1,0
3,41,1020201,20050630,0.131771,1,0,0.001485,0.078767,18.788057,0.015216,0.372252,0.362995,0.642943,0.046566,0,0.134151,1,0
4,42,1020201,20050930,0.135333,1,0,0.001715,0.080054,18.818710,0.016574,0.325251,0.376771,0.623793,0.042985,0,0.134151,1,0


In [13]:
date = pd.Categorical(data.rssd9999)
# company_code = pd.Categorical(data.rssd9001)
data = data.set_index(['rssd9001', 'rssd9999'])
data['rssd9999'] = date
# data['rssd9001'] = company_code
data['after_DFA_1*affect'] = data.after_DFA_1 * data.affect
data.head()

Unnamed: 0  bhc_avgtradingratio  treat_3_b_avg  \
rssd9001 rssd9999                                                   
1020201  20040930          38             0.126453              1   
         20041231          39             0.139026              1   
         20050331          40             0.130043              1   
         20050630          41             0.131771              1   
         20050930          42             0.135333              1   

                   after_DFA_1  dep_roa1  dep_leverage  dep_lnassets  \
rssd9001 rssd9999                                                      
1020201  20040930            0  0.002906      0.070028     18.610798   
         20041231            0  0.002053      0.074122     18.764626   
         20050331            0  0.002237      0.076953     18.768549   
         20050630            0  0.001485      0.078767     18.788057   
         20050930            0  0.001715      0.080054     18.818710   

                   dep_creditrisk_total3   dep_cir  dep_depositratio  \
rssd9001 rssd9999                                                      
1020201  20040930               0.013813  0.339097          0.390303   
         20041231               0.014689  0.414206          0.358281   
         20050331               0.015002  0.357776          0.357660   
         20050630               0.015216  0.372252          0.362995   
         20050930               0.016574  0.325251          0.376771   

                   dep_loans_REratio  dep_liquidity  dep_cpp_bankquarter  \
rssd9001 rssd9999                                                          
1020201  20040930           0.758043       0.044461                    0   
         20041231           0.647956       0.042322                    0   
         20050331           0.651144       0.045373                    0   
         20050630           0.642943       0.046566                    0   
         20050930           0.623793       0.042985                    0   

                     affect  affected_bhc  after_DFA_1*affected_bhc  rssd9999  \
rssd9001 rssd9999                                                               
1020201  20040930  0.134151             1                         0  20040930   
         20041231  0.134151             1                         0  20041231   
         20050331  0.134151             1                         0  20050331   
         20050630  0.134151             1                         0  20050630   
         20050930  0.134151             1                         0  20050930   

                   after_DFA_1*affect  
rssd9001 rssd9999                      
1020201  20040930                 0.0  
         20041231                 0.0  
         20050331                 0.0  
         20050630                 0.0  
         20050930                 0.0

In [14]:
control_vars_w_DFA_Affect = ['after_DFA_1*affect','dep_roa1', 'dep_leverage', 'dep_lnassets', 'dep_creditrisk_total3', 'dep_cir', 'dep_depositratio', 'dep_loans_REratio', 'dep_liquidity', 'dep_cpp_bankquarter']
exog_DFA_and_Affect = sm.add_constant(data[control_vars_w_DFA_Affect])

In [15]:
mod = PanelOLS(data.bhc_avgtradingratio, exog_DFA_and_Affect, entity_effects=True)
re_res = mod.fit()
print(re_res)

                           PanelOLS Estimation Summary                           
Dep. Variable:     bhc_avgtradingratio   R-squared:                        0.2227
Estimator:                    PanelOLS   R-squared (Between):             -0.3589
No. Observations:                 1580   R-squared (Within):               0.2227
Date:                 Sun, Nov 03 2019   R-squared (Overall):             -0.5522
Time:                         21:48:03   Log-likelihood                    4317.8
Cov. Estimator:             Unadjusted                                           
                                         F-statistic:                      43.083
Entities:                           66   P-value                           0.0000
Avg Obs:                        23.939   Distribution:                 F(10,1504)
Min Obs:                        1.0000                                           
Max Obs:                        38.000   F-statistic (robust):             43.083
                